AuxTel test LTS-337-027 (Auxiliary Telescope Dome Rotation Requirement)

Specification: None of the upgrades or modifications of the Auxiliary telescope shall degrade its total image quality. The total telescope image quality degradation shall not exceed the existing telescope system image quality value Aux_Tel_Tel_IQ at any point in the central 5 by 5 arcminutes of the field of view. The image quality shall be measured over a 5 minute guided exposure.

Discussion: The overall image quality budget 0.35"FWHM to the Auxiliary Telescope median delivered PSF. 

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The Auxiliary Telescope shall deliver this image quality contribution to the median delivered PSF: | 0.35       | ArcsecFWHM   |Aux_Tel_Tel_IQ|


This notebook is divided into 6 sections:
1. Notebook setup
2. 75 degree elevation IQ
3. 35 degree elevation IQ
7. Conclusion. 

# Notebook setup 

In [16]:
import numpy as np
from astropy import units as u
from matplotlib import pyplot as plt
from datetime import datetime, date
from astropy.time import Time, TimeDelta

import lsst.daf.butler as dafButler
from lsst.summit.utils import BestEffortIsr
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfConfig
from lsst.pex.exceptions import InvalidParameterError
from lsst.ts.observatory.control.constants import latiss_constants
from lsst.geom import PointD
# below is an unused import, but is required to use the `psfex` fitter
import lsst.meas.extensions.psfex.psfexPsfDeterminer # noqa: F401

%matplotlib inline

In [17]:
# Enter the filename where to save the results.
filename = 'AuxTel_LTS-337-014_' + date.today().strftime("%y%m%d") + '.txt'
print(filename)

AuxTel_LTS-337-014_220608.txt


#### Declare butler repo parameters

In [18]:
datapath='/repo/LATISS'
butler = dafButler.Butler(datapath, instrument='LATISS', collections=['LATISS/raw/all','LATISS_test_data'])
dataset='raw'
best_effort_isr = BestEffortIsr()

# Calculate Stellar PSF Information

### Declare the function that does the heavy lifting using DM tools
Note that this is very sensitive to the number of sources in the field. <br>
If there are not enough sources this will fail.

In [19]:
def measurePsf(exp):
    platescale = latiss_constants.pixel_scale

    imCharConfig = CharacterizeImageTask.ConfigClass()
    imCharConfig.doMeasurePsf = True
    imCharConfig.useSimplePsf = True
    
    imCharConfig.doApCorr = False
    imCharConfig.doDeblend = False
    
    installConfig = InstallGaussianPsfConfig()
    exp.setPsf(None)  # if not set to none, fwhm max para is ignored
    installConfig.fwhm = 15
    installConfig.width = 61
    
    imCharConfig.installSimplePsf = installConfig    
    
    imCharConfig.detection.includeThresholdMultiplier = 5
    imCharConfig.measurePsf.starSelector['objectSize'].doFluxLimit = True
    imCharConfig.measurePsf.starSelector['objectSize'].fluxMin = 12500.0
    imCharConfig.measurePsf.starSelector['objectSize'].fluxMax = 0.0
    imCharConfig.measurePsf.starSelector['objectSize'].doSignalToNoiseLimit = False
    imCharConfig.measurePsf.starSelector['objectSize'].signalToNoiseMin = 20.0
    imCharConfig.measurePsf.starSelector['objectSize'].signalToNoiseMax = 0.0
    imCharConfig.measurePsf.starSelector['objectSize'].widthMin = 0.0
    imCharConfig.measurePsf.starSelector['objectSize'].widthMax = 80.0  # default 10
    imCharConfig.measurePsf.starSelector['objectSize'].sourceFluxField = "base_GaussianFlux_instFlux"
    imCharConfig.measurePsf.starSelector['objectSize'].widthStdAllowed = 0.15 # 0.15 default
    imCharConfig.measurePsf.starSelector['objectSize'].nSigmaClip = 2.0
    
    imCharConfig.measurePsf.psfDeterminer.name = 'psfex'
    imCharConfig.measurePsf.psfDeterminer['psfex'].spatialOrder = 1
    imCharConfig.psfIterations=1
    
    imCharConfig.background.binSize = 2000
    imCharConfig.background.approxOrderX = 2

    imCharConfig.detection.background = imCharConfig.background
    
    imCharTask = CharacterizeImageTask(config=imCharConfig)

    result = imCharTask.run(exp)

    psf = exp.getPsf()
    ixx = psf.computeShape(exp.getBBox().getCenter()).getIxx()
    iyy = psf.computeShape(exp.getBBox().getCenter()).getIyy()
    psfShape = psf.computeShape(exp.getBBox().getCenter()).getDeterminantRadius()
    
    fwhmX = np.sqrt(ixx)*2.355*platescale
    fwhmY = np.sqrt(iyy)*2.355*platescale
    
    overallFwhm = psfShape * 2.355 * platescale
    print(f"Psf shape from imChar task (x,y) = ({fwhmX:.3f}, {fwhmY:.3f}) FWHM arcsec")
    return fwhmX, fwhmY, overallFwhm, psf

### Declare QuickFrameMeasurement tasks
These are run if the measurePsf method fails. It provides a reasonable yet less-accurate representation.

In [20]:
qm_config = QuickFrameMeasurementTask.ConfigClass()
qm = QuickFrameMeasurementTask(config=qm_config)

### Select the data ID of choice

In [42]:
fwhmX=None; fwhmY=None; overallFwhm=None
result=None
psf=None
success=False
line=[]

for i in np.arange(135, 146, dtype=int):
    dataId = {'day_obs': 20220526, 'seq_num': i.item(), 'detector': 0, "instrument": 'LATISS'}
    print(dataId)
    best_effort_isr.doWrite = False  # Don't write to butler database
    exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
    
    # Run the methods to get the psf data
    try:
        fwhmX, fwhmY, overallFwhm, psf = measurePsf(exp)
        success=True
        pass
    except InvalidParameterError as e:
        print('Caught the InvalidParameterError, measurePsf was not successful')
        pass
    except RuntimeError as e:
        print('Caught the RuntimeError, measurePsf was not successful')
        pass

    if not success:
        print('Using Merlin\'s simplified algorithm')
        result = qm.run(exp)
        if result.success:
            fwhmX=result.medianXxYy[0]
            fwhmY=result.medianXxYy[1]
            overallFwhm=np.sqrt(result.medianXxYy[0]**2 + result.medianXxYy[1]**2)
        else:
            raise RuntimeError('No PSF could be derived using either method')
    
    
    # Derive corrections for airmass and wavelength
    filter_band=exp.getInfo().getFilterLabel().bandLabel 
    airmass=((exp.getInfo().getMetadata().get('AMSTART')+exp.getInfo().getMetadata().get('AMEND'))/2.0)
    elevation=((exp.getInfo().getMetadata().get('ELSTART')+exp.getInfo().getMetadata().get('ELEND'))/2.0)
    dimm_seeing=exp.getInfo().getMetadata().get('SEEING')
    
    
    airmass_corr_fwhm = overallFwhm * (airmass ** -0.6)
    
    filter_wavelength= {'white': 700, 'g': 475, 'r': 623, 'i': 763, 'z': 828.0}
    corr_fwhm = airmass_corr_fwhm * ((500. / filter_wavelength[filter_band]) ** -0.2)

    
    # Build each PD entry 
    line.append(
        {'exposure': i,
         'fwhmX': fwhmX,
         'fwhmY': fwhmY, 
         'overallFwhm':overallFwhm,  
         'airmass': airmass,
         'filter_band' : filter_band,
         'dimm_seeing': dimm_seeing,
         'corr_fwhm': corr_fwhm
        }
    )
    
df = pd.DataFrame(line)   

{'day_obs': 20220526, 'seq_num': 135, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.432, 1.481) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 136, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.360, 1.443) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 137, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.274, 1.325) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 138, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.337, 1.393) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 139, 'detector': 0, 'instrument': 'LATISS'}


...

Psf shape from imChar task (x,y) = (1.394, 1.454) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 142, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.410, 1.447) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 143, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.681, 1.659) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 144, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.441, 1.566) FWHM arcsec
{'day_obs': 20220526, 'seq_num': 145, 'detector': 0, 'instrument': 'LATISS'}


Psf shape from imChar task (x,y) = (1.421, 1.536) FWHM arcsec


In [43]:
df.to_csv('AuxTel_LTS-337-014_220608b.txt')

In [44]:
df

,exposure,fwhmX,fwhmY,overallFwhm,airmass,filter_band,dimm_seeing,corr_fwhm
0,135,1.432213,1.480916,1.456356,1.550099,r,1.019236,1.169919
1,136,1.360024,1.443089,1.400682,1.550136,r,1.224519,1.125178
2,137,1.273828,1.324774,1.298891,1.550173,r,1.224519,1.043393
3,138,1.337126,1.393243,1.359064,1.550213,r,0.992373,1.091713
4,139,1.319926,1.347844,1.333697,1.550255,r,0.992373,1.071320
5,140,1.417474,1.499312,1.456285,1.550298,r,1.016877,1.169771
6,141,1.394170,1.453657,1.423504,1.550344,r,1.016877,1.143419
7,142,1.409938,1.446995,1.428132,1.550392,r,1.027855,1.147115
8,143,1.681227,1.659198,1.669394,1.550441,r,1.102542,1.340878
9,144,1.440937,1.566001,1.502029,1.550493,r,1.102542,1.206424


****

# Conclusion

The dome azimuth rotation speed has been examined in 3 dedicated rotation tests of different rotation angles (210, 90 and 30 degrees) and an offline analysis of an observing night, to replicate the dome operations cadence.
In none of these test cases, the azimuth rotational speed has been above the minimum 4 deg/sec speed requirement from LTS-337-030 (Auxiliary Telescope Dome Rotation Requirement). 
